In [0]:
import re
import pandas as pd
import numpy as np
import spacy
import re
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.pipeline import SentenceSegmenter
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.pipeline import SentenceSegmenter
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from keras.layers import Dense, Embedding, Input
from keras.models import Model
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.layers import Input, concatenate # new! 
from pickle import dump,load
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding # new!
from keras.callbacks import ModelCheckpoint # new! 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
#import tensorflow.keras.preprocessing.text.Tokenizer
import os # new! 
import tensorflow as tf 

Making sure the GPU is working

In [4]:
import tensorflow as tf 
tf.test.gpu_device_name()

'/device:GPU:0'

Mounting the Drive

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


Fetching the data

In [6]:
clean_data = pd.read_csv('/gdrive/My Drive/amazon_review_full_csv/cleaned_topic_amazon_reviews.csv')
clean_data.head()

,Unnamed: 0,rating,title,review,review_category,company,clean_review,topic_category,topic_name
0,0,1,Buyer beware,This is a selfpublished book and if you want t...,Bad,Amazon,this selfpublished book want know whyread para...,1,books
1,1,1,Another Abysmal Digital Copy,Rather than scratches and insect droppings thi...,Bad,Amazon,rather scratch insect dropping random pixelati...,5,video-quality
2,2,5,CHECK OUT THE K6000 CHARGER!,Greetings Kodak also has a K6000 charger which...,Good,Amazon,greetings kodak k6000 charger come nimh mah ba...,11,electronics
3,3,2,Maybe I got a bad one.,My experience: I got this from FM not Amazon T...,Bad,Amazon,my experience get fm amazon the dvd player rat...,5,video-quality
4,4,2,Didn't live up to expectations.,I ordered these shoes to replace the previous ...,Bad,Amazon,order shoe replace previous pair rip apart sol...,4,order


In [0]:
clean_data_grouped = clean_data.groupby(['topic_name','review_category']).agg(
  {

    'review_category':'count'

  })



In [8]:
clean_data_grouped= clean_data_grouped.unstack().reset_index()
clean_data_grouped

topic_name review_category             
review_category                              Average    Bad  Good
0                              books            2058   6663  4118
1                 christmas shopping             763   3347  2173
2                        electronics             382   1867   698
3                              games             329   1010   797
4                             movies             858   2218  1750
5                              music            1320   3404  3375
6                              order             881   4393  1412
7                            packing            3596  11887  7048
8                product-information            1273   6078  2148
9                  refund-and-return            1408   7471  1424
10                           reviews             817   3459  1348
11                          shipping            1480   2270  6621
12                         tv-series             612   1746  1699
13                     video-quality             913   3464  1360

In [0]:
def spacy_preprocessing(text):
    clean_text=[]
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'.", "", text)
    pos_tag= ['NUM','PRON']
    for token in nlp(text):
       #print(token.text), #(token.lemma_ != "-PRON-") & 
       if (token.lemma_ != "-PRON-") & (token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n\'...\'') & (token.text not in nlp.Defaults.stop_words) & (token.pos_ not in pos_tag) :
           clean_text.append(token.lemma_.lower())
       elif (token.lemma_ == "-PRON-")  & (token.text not in nlp.Defaults.stop_words):
           clean_text.append(token.text.lower())
       else:
           continue
    return " ".join(clean_text)

In [9]:
clean_data_grouped['percentage_bad']= clean_data_grouped['review_category']['Bad']/(clean_data_grouped['review_category']['Average']+clean_data_grouped['review_category']['Good']+clean_data_grouped['review_category']['Bad'])
clean_data_grouped=clean_data_grouped.sort_values(by=['percentage_bad'], ascending=False)
clean_data_grouped

topic_name review_category  ...       percentage_bad
review_category                              Average  ...  Good               
9                  refund-and-return            1408  ...  1424       0.725129
6                              order             881  ...  1412       0.657045
8                product-information            1273  ...  2148       0.639857
2                        electronics             382  ...   698       0.633526
10                           reviews             817  ...  1348       0.615043
13                     video-quality             913  ...  1360       0.603800
1                 christmas shopping             763  ...  2173       0.532707
7                            packing            3596  ...  7048       0.527584
0                              books            2058  ...  4118       0.518966
3                              games             329  ...   797       0.472846
4                             movies             858  ...  1750       0.459594
12                         tv-series             612  ...  1699       0.430367
5                              music            1320  ...  3375       0.420299
11                          shipping            1480  ...  6621       0.218880

[14 rows x 5 columns]

In [10]:
filtered_data = clean_data[clean_data['topic_name'].isin(['movies','music','books'])]
filtered_data.head()

,Unnamed: 0,rating,title,review,review_category,company,clean_review,topic_category,topic_name
0,0,1,Buyer beware,This is a selfpublished book and if you want t...,Bad,Amazon,this selfpublished book want know whyread para...,1,books
8,8,1,ADDONICS PORTABLE CD DRIVE - I am disappointed...,I am disappointed in its performance It seems ...,Bad,Amazon,disappoint performance underpowered constantly...,3,music
17,17,5,Cruel & Unusual,Received the book promptly and in great condit...,Good,Amazon,receive book promptly great condition thank am...,1,books
18,18,5,Great book.,Not as spell binding as others in this series ...,Good,Amazon,not spell bind series good take get couple cha...,1,books
19,19,4,Author's comments,As the author of this book I would just like t...,Good,Amazon,as author book like successfully publish uk re...,1,books


# prepare text samples and their labels



In [0]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [0]:
for column in ['review_category','topic_name']:
 filtered_data= create_dummies(filtered_data, column)

In [13]:
filtered_data

,Unnamed: 0,rating,title,review,review_category,company,clean_review,topic_category,topic_name,review_category_Average,review_category_Bad,review_category_Good,topic_name_books,topic_name_movies,topic_name_music
0,0,1,Buyer beware,This is a selfpublished book and if you want t...,Bad,Amazon,this selfpublished book want know whyread para...,1,books,0,1,0,1,0,0
8,8,1,ADDONICS PORTABLE CD DRIVE - I am disappointed...,I am disappointed in its performance It seems ...,Bad,Amazon,disappoint performance underpowered constantly...,3,music,0,1,0,0,0,1
17,17,5,Cruel & Unusual,Received the book promptly and in great condit...,Good,Amazon,receive book promptly great condition thank am...,1,books,0,0,1,1,0,0
18,18,5,Great book.,Not as spell binding as others in this series ...,Good,Amazon,not spell bind series good take get couple cha...,1,books,0,0,1,1,0,0
19,19,4,Author's comments,As the author of this book I would just like t...,Good,Amazon,as author book like successfully publish uk re...,1,books,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111928,111928,4,I love vampires.,I will admit I am biased I like almost any vam...,Good,Amazon,admit biased like vampire movie tv love buffy ...,2,movies,0,0,1,0,1,0
111929,111929,4,reading,these book has a lot of different learning act...,Good,Amazon,book lot different learn activie word like pig...,1,books,0,0,1,1,0,0
111930,111930,2,Uninspired,Firstly the tracklist for this single does not...,Bad,Amazon,firstly tracklist single match cover art pictu...,3,music,0,1,0,0,0,1
111931,111931,5,Great!,I read this book when I was a teenager and fee...,Good,Amazon,read book teenager feel n love low character w...,1,books,0,0,1,1,0,0


In [14]:
filtered_data.columns

Index(['Unnamed: 0', 'rating', 'title', 'review', 'review_category', 'company',
       'clean_review', 'topic_category', 'topic_name',
       'review_category_Average', 'review_category_Bad',
       'review_category_Good', 'topic_name_books', 'topic_name_movies',
       'topic_name_music'],
      dtype='object')

In [15]:
print('Loading in comments...')
print(filtered_data["clean_review"].dtype)
print("Transforamation from pandas colums to numpy array using .values")
sentences = filtered_data["clean_review"].fillna("DUMMY_VALUE").values
possible_labels = ['review_category_Average', 'review_category_Bad',
       'review_category_Good', 'topic_name_books', 'topic_name_movies',
       'topic_name_music']
targets = filtered_data[possible_labels].values
print(type(sentences))
print(type(targets))
print((sentences[0]))
print((targets[0]))

Loading in comments...
object
Transforamation from pandas colums to numpy array using .values
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
this selfpublished book want know whyread paragraph those star review write ms haddon family friendsor imagine read thingi spend evening book friend hysteric read bit piece definitely bad enter kind bad book contest believe amazon sell kind thing maybe offer 8th grade term paper to kill mockingbird"a book sure ms haddon hear anyway mood send book jokestay far far away
[0 1 0 1 0 0]


**Setting up initial configuration**

In [0]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
output_dir = '/gdrive/My Drive/amazon_review_full_csv'

Tokenizing the data

In [0]:
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [18]:
sequences[1:3]

[[231, 421, 1450, 46, 8, 5, 258, 12, 8627, 29, 46, 73, 2],
 [68, 1, 1388, 13, 189, 103, 2, 1060, 6, 1]]

In [0]:
#word_seq = [[idx2word[i] for i in sequences[j]] for j in range(len(sequences))]
#word_seq[1:3]

Creating Dictionary for word-integer mapping

In [20]:
idx2word = tokenizer.index_word
#word2idx = {k:v for v,k in idx2word.items()}
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))
print(word2idx['instead'])
print(idx2word[157])

Found 71701 unique tokens.
157
instead


Finding out the Max Sequence Lenght and  pad sequences so that we get a N x T matrix

In [21]:
seq_len = [len(sent) for sent in sequences]
max_seq_len = max(seq_len)
max_seq_len

133

In [22]:
padded_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', padded_data.shape)

Shape of data tensor: (25764, 100)


In [23]:
padded_data[5]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,   525,  1600,   183,     8,   148,
           4,   112,   307,    33,  1612,    88,    31,   859,   130,
          28,    72,    11,   112,    23,   325,   115,    31,  1085,
         136,   560,  1001,   139,    41,     4,    16,     8,    63,
           4,   365,   541,    16,     4,   285,   315,    26,     9,
        3834,  5349,     4,    26,     9,  3834,  5349,     4,     7,
         141,    10,     4,   184,  9483,   153,    10,     7,     6,
        5349,    78,  1778,  2658,   199,   727,    53,  3834,  5349,
       10617,    72,   220,     4,  1045,   155,   296,  1382,  1237,
         148], dtype=int32)

Creating the mebedding Matrix from Glove

In [24]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('/gdrive/My Drive/amazon_review_full_csv/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
  for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:] , dtype='float32')
    word2vec[word]=value
print(len(word2vec))

Loading word vectors...
400000


In [25]:
# prepare embedding matrix
#print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
print(num_words)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
embedding_matrix
print((embedding_matrix))
print(type(embedding_matrix))
print(embedding_matrix.shape)

20000
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
(20000, 50)


Filling the empty embedding matrix with the vector values from the Glove for the vocublary and vocubalry size of our dataset

In [26]:

for word,i in word2idx.items():
  #print(word)
  #print(i)
  if i < MAX_VOCAB_SIZE:
    embedding_vector= word2vec.get(word)
    if embedding_vector is not None:
      embedding_matrix[i]=embedding_vector
print(embedding_matrix.shape)
print((embedding_matrix))

(20000, 50)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.0076543   0.93456    -0.73189002 ... -0.089847   -0.27223
  -0.0088736 ]
 [ 0.56818998 -0.95608997  0.51209003 ... -0.16356     0.059135
  -0.76425999]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.59521002 -0.82484001 -0.88629001 ...  0.39816999 -0.25731
   0.40994   ]
 [ 0.48909    -0.071652    0.44681001 ... -0.98063999  0.29967001
  -0.56155002]]


Creating an Embedding Layer

In [27]:
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

Building the model now

In [28]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
embeddings = embedding_layer(input_)
drop_embed_layer = SpatialDropout1D(.2, name='drop_embed')(embeddings)

conv1 = Conv1D(256, 20,strides=1, activation='relu')(drop_embed_layer)
maxp_1 = GlobalMaxPooling1D(name='maxp_1')(conv1)

conv2= Conv1D(256, 10, activation='relu' )(drop_embed_layer)
maxp_2 = GlobalMaxPooling1D(name='maxp_2')(conv2)

conv3= Conv1D(256, 5, activation='relu' )(drop_embed_layer)
maxp_3 = GlobalMaxPooling1D(name='maxp_3')(conv3)

concat = concatenate([maxp_1, maxp_2, maxp_3])

dense_layer = Dense(64, activation='relu', name='dense')(concat)


output = Dense(len(possible_labels), activation="sigmoid")(dense_layer)
#output = dense(dense_layer)
model = Model(input_, output)

model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(lr=0.01),
  metrics=['accuracy']
)
print(model.summary())














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 50)      1000000     input_1[0][0]                    
__________________________________________________________________________________________________
drop_embed (SpatialDropout1D)   (None, 100, 50)      0           embedding_1[0][0]                
______________________________________________________________________

In [29]:
model.fit(padded_data,
        targets,
        batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)  
          



Train on 20611 samples, validate on 5153 samples
Epoch 1/10
20611/20611 [==============================] - 4s 209us/step - loss: 0.4558 - acc: 0.8124 - val_loss: 0.3082 - val_acc: 0.8550
Epoch 2/10
20611/20611 [==============================] - 3s 133us/step - loss: 0.2799 - acc: 0.8704 - val_loss: 0.2765 - val_acc: 0.8772
Epoch 3/10
20611/20611 [==============================] - 3s 136us/step - loss: 0.2615 - acc: 0.8823 - val_loss: 0.2717 - val_acc: 0.8753
Epoch 4/10
20611/20611 [==============================] - 3s 136us/step - loss: 0.2467 - acc: 0.8901 - val_loss: 0.2718 - val_acc: 0.8750
Epoch 5/10
20611/20611 [==============================] - 3s 135us/step - loss: 0.2352 - acc: 0.8955 - val_loss: 0.3171 - val_acc: 0.8553
Epoch 6/10
20611/20611 [==============================] - 3s 136us/step - loss: 0.2284 - acc: 0.8996 - val_loss: 0.2805 - val_acc: 0.8743
Epoch 7/10
20611/20611 [==============================] - 3s 136us/step - loss: 0.2158 - acc: 0.9065 - val_loss: 0.4763 -

Saving the model

In [0]:
model.save(output_dir+"/weights_parallel_conv.01.hdf5")

In [0]:
model.load_weights(output_dir+"/weights_parallel_conv.01.hdf5")

In [34]:
print(len(padded_data))

25764


In [0]:
y_hat = model.predict(padded_data.reshape(25764,100))

In [37]:
y_hat.shape

(25764, 6)

In [38]:
y_hat.round(2)

array([[0.22, 0.81, 0.01, 1.  , 0.  , 0.  ],
       [0.1 , 0.94, 0.04, 0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  , 1.  , 0.  , 0.  ],
       ...,
       [0.28, 0.07, 0.78, 0.  , 0.  , 1.  ],
       [0.05, 0.24, 0.47, 1.  , 0.  , 0.  ],
       [0.05, 0.02, 0.98, 0.  , 0.  , 1.  ]], dtype=float32)

In [39]:
targets[:,1].shape

(25764,)

In [40]:
aucs = []
for j in range(6):
    auc = roc_auc_score(targets[:,j], y_hat[:,j])
    aucs.append(auc)
print(np.mean(aucs))

0.9508196367227838


In [0]:
music_test = "this piece of music is the shittiest i have seen so bad, the lyrics were horrible !!"

In [0]:
def predict_class(text):
  cleane_text = spacy_preprocessing(text)
  text2idx = tokenizer.texts_to_sequences([cleane_text])
  padded_seq = pad_sequences(text2idx, maxlen=MAX_SEQUENCE_LENGTH, truncating='pre')
  yhat= model.predict(padded_seq)
  yhat = yhat.round(2)
  return yhat

In [0]:
predicted_class = predict_class(music_test)
predicted_class_grade= predicted_class[0,0:3].argsort()[-3:]
predicted_class_category= predicted_class[0,3:6].argsort()[-3:]

In [127]:
predicted_class_grade  ##1 means bad so the prediction was correct 

array([2, 0, 1])

In [129]:
predicted_class_category ###2 means Music here  so the prediction was correct 

array([0, 1, 2])